## 豆瓣电影聚类分析

### 作业要求：（单人组）
##### Task1：数据预处理和特征向量化。在这个任务中，我们需要对原始数据进行预处理，将文本数据转换为特征向量。

需要完成：
- 读取数据：从给定的数据文件(movies.xls)中读取电影数据。（代码已给出）
- 清洗数据：处理缺失值、异常值等，对“简介”中的中文数据进行分词，英文数据进行清洗。
- 使用TF-IDF对“类型”和“标签”进行向量化：将文本类型和标签数据转换为数值特征向量，并用PCA降维
- 使用Word2Vec对分词后的“简介”进行向量化：将电影简介数据转换为数值特征向量。

##### Task2：特征融合和降维。在这个任务中，我们需要将不同的特征融合在一起，并使用PCA降维。

需要完成：
- 归一化数值特征：对数值特征进行归一化处理，使其在相同的取值范围内。
- 特征融合：将不同的特征向量组合成一个整体特征向量（可认为是每个电影的Embedding向量）。（建议特征包含：TF-IDF结果、WV结果、电影年份、豆瓣评分，使用np拼接即可）
- PCA降维：通过主成分分析（PCA）对特征向量进行降维处理。（可设置设置累积方差贡献率阈值确定留下多少主成分）

##### Task3：K-means聚类。在这个任务中，我们需要使用K-means聚类方法对降维后的特征向量进行聚类。

需要完成：
- 肘部法确定最佳簇数量：通过计算不同簇数量下的误差平方和，确定最佳簇数量。
- 使用最佳簇数量进行K-means聚类：根据最佳簇数量，对降维后的特征向量进行聚类。
- 分析聚类结果：输出每个簇的代表电影、簇中电影的数量、评分分布等，也可对结果进行可视化展示。


##### Task4：基于电影的 Embedding向量和表中其他信息，为导演和演员生成 Embedding，并进行无监督分类。然后使用可视化的方法，分析2-3个属于不同类别导演的特点

需要完成：
- 计算导演和演员的 Embedding
- 对导演、演员分别进行无监督分类（最好不要用K-means，可以尝试层次聚类、DBSCAN、GMM等方法）
- 分析聚类结果：输出每个类的代表人物、每个类的人数、导演电影数分布等，也可对结果进行可视化展示。
- 使用可视化的方法，分析2-3个属于不同类型的导演、与2-3个属于不同类型的演员的特点

**注意：**
- 为了使结果可复现，请尽量在所有需要随机性的地方都设置随机数种子。若实在无法稳定复现，则应当保留好所有输出。
- 可以根据自己的理解，在task1-3中使用要求之外的数据项，鼓励在过程中多进行可视化。可以在任何地方添加新的ipynb cell



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
import seaborn as sns
import jieba
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
data = pd.read_excel('./data/movies.xls')
data

,MOVIE_ID,NAME,ALIAS,ACTORS,COVER,DIRECTORS,DOUBAN_SCORE,DOUBAN_VOTES,GENRES,IMDB_ID,...,MINS,OFFICIAL_SITE,REGIONS,RELEASE_DATE,SLUG,STORYLINE,TAGS,YEAR,ACTOR_IDS,DIRECTOR_IDS
0,26670818,情定河州,情定临夏天使然,王博/吴佳尼/王姬/高丽雯/郭力行/尹哲/沈丹萍/罗中旭/臧金生/罗刚/居文沛/阎青妤,NaN,尹哲,0.0,0,剧情/爱情,NaN,...,0,NaN,中国大陆,NaT,RmNQQeyzb,电影《情定临夏天使然》讲述临夏新一代青年人发奋图强、借助国家一带一路战略励志创业的故事。,甘肃/临夏/伊斯兰/中国/2016/中国大陆/烂片/宣传伊斯兰教的电影,2049,王博:|吴佳尼:1313262|王姬:1275275|高丽雯:1325661|郭力行:135...,尹哲:1326188
1,25815002,我不是李小龙,NaN,谷尚蔚/吴孟达/曾志伟/杜海涛,NaN,洪金宝,0.0,0,动作/爱情,NaN,...,0,NaN,中国大陆,NaT,EZnVfiNYf,桀骜不驯的如龙武功高强，在一场比赛中，被打成重伤，被诊今生不能再用功夫。女友荆兰为激发他重新...,穿越/华语,2049,谷尚蔚:1330813|吴孟达:1016771|曾志伟:1002862|杜海涛:1313024,洪金宝:1055887
2,26392287,曼哈顿中国女孩,NaN,NaN,NaN,NaN,0.0,0,剧情,NaN,...,0,NaN,中国大陆 / 美国,NaT,NuUvEJnzb,平民女孩李莉只身初入曼哈顿求学，在历经迷失与困惑之后，凭借努力与善良收获了事业上的成功，同时...,NaN,2049,NaN,NaN
3,26695995,绿毛水怪,NaN,NaN,NaN,梁栋/吴国墉,0.0,0,爱情,NaN,...,0,NaN,中国大陆,NaT,rqaqyb6ea,王小波经典中篇小说《绿毛水怪》将改编电影。《绿毛水怪》是王小波早期手稿作品，以天马行空的想象...,小波/王小波/爱情/小说改编/文学改编/剧情/中国/2017,2049,NaN,梁栋:|吴国墉:
4,26392292,为了祖国,NaN,NaN,NaN,NaN,0.0,0,剧情/历史,NaN,...,0,NaN,中国大陆 / 韩国,NaT,Inrqy7uib,1932年上海虹口爆炸案后，韩国国父金九在褚辅成、朱爱宝这些普通群众的帮助下逃到嘉兴避难。金...,NaN,2049,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38184,1507152,德州电锯杀人狂前传,德州电锯杀人狂：从头开始 / 德州链锯杀人狂前传 / 德州电锯杀人狂——成魔之路 / 德州链...,乔丹娜·布鲁斯特/安德鲁·布莱尼亚斯基/迪奥拉·拜尔德/泰勒·汉德雷/李·厄米/马特·波莫,https://img3.doubanio.com/view/photo/s_ratio_p...,乔纳森·里贝斯曼,6.8,13856,惊悚/恐怖,tt0420294,...,91,NaN,美国,2006-10-06,QIJRYBeEf,在德州某屠宰场，肥胖的女工正在案板切肉，突然感到腹中剧痛，原来身怀有孕的她羊水已破，产下一个...,血腥/恐怖/美国/惊悚/暴力/德洲电锯杀人狂前传/美国电影/2006,2006,乔丹娜·布鲁斯特:1000036|安德鲁·布莱尼亚斯基:1004677|迪奥拉·拜尔德:10...,乔纳森·里贝斯曼:1049991
38185,1510432,麻雀变王妃2,王子与我2,卢克·梅伯利/乔纳森·佛斯/玛瑞·亚达波,https://img1.doubanio.com/view/photo/s_ratio_p...,NaN,5.8,2734,喜剧/爱情,tt0477072,...,0,NaN,美国,2006-02-21,YR2z72b6e,皇家婚禮是2003年浪漫喜劇片麻雀變王妃的續集 在前集中 珮姬愛上愛得華 卻不知他是如假包換...,爱情/美国/美国电影/麻雀变王妃/喜剧/浪漫喜剧/童话/电影,2006,卢克·梅伯利:1031880|Kam Heskin:1056143|Clemency Bur...,NaN
38186,1874845,暴力城市,暴力城 / 伙伴 / 同伴,安在模/郑斗洪/金秉玉/金花雨/金瑞亨/金时厚/李凡秀/温宙完/柳昇完,https://img1.doubanio.com/view/photo/s_ratio_p...,柳昇完,6.2,552,剧情/动作/犯罪,tt0821470,...,92,NaN,韩国,2006-05-25,27uybMuUf,首尔重案组刑警郑泰秀（郑斗洪 饰）接到好友吴汪才的死讯，于是赶回十年未曾亲近的故乡。葬礼上，...,韩国/动作/黑帮/暴力/韩国电影/犯罪/2006/柳承完,2006,安在模:1051565|郑斗洪:1109833|金秉玉:1319357|金花雨:131576...,柳昇完:1138335
38187,2075992,错身姐妹,NaN,露·杜瓦隆/斯蒂芬·瑞/威廉·B·戴维斯/科洛·塞维尼,https://img1.doubanio.com/view/photo/s_ratio_p...,道格拉斯、巴克,5.4,117,悬疑/惊悚/恐怖/犯罪,tt0486652,...,92,NaN,美国 / 加拿大 / 英国,2010-10-29,zjFBmFRAe,A pair of Siamese twins are separated and one ...,惊悚/美国/悬疑/恐怖/我看过的英语电影/我看过的电影/我看过的恐怖电影/美国电影,2006,露·杜瓦隆:1050180|斯蒂芬·瑞:1041170|威廉·B·戴维斯:1014073|科...,道格拉斯、巴克:


In [3]:
# 标签、类型数据split、清理


In [4]:
# 使用TF-IDF对“类型”和“标签”进行向量化


In [5]:
# 使用TruncatedSVD对向量化后的数据进行降维（用时较久，约2分钟，初始化TruncatedSVD时设置random_state参数）


In [6]:
# 电影简介文本数据清洗，中文数据分词


In [7]:
#电影简介文本数据向量化(Word2Vec，初始化Word2Vec时设置seed参数)


In [8]:
# 归一化数值特征


In [9]:
# 特征融合(使用np进行拼接即可)


In [10]:
# PCA降维（可设置设置累积方差贡献率阈值确定留下多少主成分）提示：可将n_components参数设置为None，计算出所有主成分，然后根据累积方差贡献率阈值确定留下多少主成分


In [11]:
# 肘部法确定最佳簇数量并进行聚类，可进行cluster-inertia可视化。使用KMeans时设置random_state参数。提示：kmeans实例有inertia_变量可用于获取聚类的簇内误差平方和


In [12]:
# 打印每个簇的信息、可视化等

In [13]:
# Task 4:自由发挥
